In [ ]:
!pip install proglearn
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
!unzip /content/ISIC2018_Task1-2_Training_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
!unzip /content/ISIC2018_Task1_Training_GroundTruth.zip

Streaming output truncated to the last 5000 lines.
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000225.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000226.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000227.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000228.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000229.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000230.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000231.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000232.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000233.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000234.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000235.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000236.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000239.jpg  
 extracting: ISIC2018_Task1-2_Training_Input/ISIC_0000240.jpg  
 extracting: ISIC2018_Task1-2_Training_Input/ISIC_000

In [ ]:
import os
import sys
import random

import numpy as np
import cv2
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

from proglearn import LifelongClassificationNetwork
from proglearn import NeuralClassificationTransformer
from proglearn import KNNClassificationVoter
from proglearn import SimpleArgmaxAverage

ModuleNotFoundError: ignored

In [ ]:
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

In [ ]:
image_size = 128
path = "./"
train_path = "./ISIC2018_Task1-2_Training_Input"
epochs = 1
batch_size = 19
num_tasks = 10
num_batches = 10

## Training Ids
train_ids = []
for files in os.walk(train_path):
   for name in files:
      train_ids.append(name)

train_ids = train_ids[2]
train_ids = [i.split('.', 1)[0] for i in train_ids]
val_data_size = 171

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(concat)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

def bottleneck(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(x)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides, activation="relu")(c)
    return c

In [ ]:
def UNet():
    f = [16, 32, 64, 128, 256]
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) 
    c2, p2 = down_block(p1, f[1]) 
    c3, p3 = down_block(p2, f[2])
    c4, p4 = down_block(p3, f[3]) 
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
model = UNet()

default_transformer_kwargs = {
                "network": model,
                "euclidean_layer_idx": -2,
                "loss": "categorical_crossentropy",
                "optimizer": Adam(3e-4),
                "fit_kwargs": {
                    "epochs": num_epochs,
                    "callbacks": [EarlyStopping(patience=5, monitor="val_loss")],
                    "verbose": False,
                    "validation_split": 0.33,
                    "batch_size": batch_size,
                },
            }
default_voter_class = MLKNNClassificationVoter
learner = ProgressiveLearner(
                default_transformer_class=default_transformer_class,
                default_transformer_kwargs=default_transformer_kwargs,
                default_voter_class=default_voter_class,)

In [ ]:
def test_predict_without_fit(image_path, lcn):
    X = np.array(cv2.imread(img_path, 0))
    X = np.tile(X, (100, 1, 1))
    lcn.predict(X, task_id=10)

In [ ]:
# intentially trigger Not Fitted Error
img_path = 'ISIC2018_Task1-2_Training_Input/ISIC_0013494.jpg'
test_predict_without_fit(img_path, lcn)

In [ ]:
def load_data(path, ids, size):
    image_path = [os.path.join(path, "ISIC2018_Task1-2_Training_Input", id) + ".jpg" for id in ids]
    mask_path = [os.path.join(path, "ISIC2018_Task1_Training_GroundTruth", id) + "_segmentation.png" for id in ids]


    X = tf.convert_to_tensor([cv2.resize(cv2.imread(img_path,1), (size, size))/255.0 for img_path in image_path])
    Y = tf.covnvert_to_tensor([cv2.resize(cv2.imread(img_path,1), (size, size))/255.0 for img_path in image_path])
    return X, Y

In [ ]:
task_train_ids = random.sample(train_ids, (num_batches + 1) * batch_size * num_tasks)

accuracies = np.zeros((num_tasks))
dices = np.zeros((num_tasks))
for task in range(0, num_tasks):

    X_train, Y_train = load_data(train_path, task_train_ids[task*(num_batches+1):(task+1)*num_batches], image_size)
    X_test, Y_test = load_data(train_path, task_train_ids[task*(num_batches+1):(task+1)*(num_batches+1)], image_size)

    learner.add_task(
                    X=X_train,
                    y=Y_train,
                    task_id=task,
                    num_transformers=1,
                    transformer_voter_decider_split=[0.67, 0.33, 0],
                )
    pred = learner.predict(
                    X_test,
                    task_id=task,
    )
    accuracy[task] = tf.mean(pred == Y_test)

In [ ]:
## Save the Weights
model.save_weights("UNetW1.h5")

In [ ]:
model.load_weights('UNetW1.h5')
model.compile()

In [ ]:
dsum = 0
total = 0
for i in range(50000):
    x, y = valid_gen.__getitem__(i)
    seg = model.predict(x)
    dice = np.sum(seg[y==1])*2.0 / (np.sum(seg) + np.sum(y))
    dice = np.sum(seg[y==1])*2.0 / (np.sum(seg) + np.sum(y))
    dsum += dice
    total = i

In [ ]:
print('# Test samples =', total)
print('Dice coefficient =', dsum / total)

In [ ]:
dsum = 0
total = 0
for i in range(50000):
    try:
        x, y = valid_gen.__getitem__(i)
        seg = model.predict(x)
        dice = np.sum(seg[y==1])*2.0 / (np.sum(seg) + np.sum(y))
        dice = np.sum(seg[y==1])*2.0 / (np.sum(seg) + np.sum(y))
        dsum += dice
    except:
        total = i

print('# Test samples =', total)
print('Dice coefficient =', dsum / total)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Expr object at 0x7efb00b77210>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Unknown node type <gast.gast.Expr object at 0x7efb00b77210>
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [ ]:
for i in range(0,20):
    x, y = valid_gen.__getitem__(i)
    result = model.predict(x)
    
    result = result > 0.5
    fig = plt.figure(figsize = (16,8))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)
    
    ax = fig.add_subplot(1, 3, 1)
    ax.imshow(x[0])
    ax = fig.add_subplot(1, 3, 2)
    ax.imshow(np.reshape(y[0]*255, (image_size, image_size)), cmap="gray")

    ax = fig.add_subplot(1, 3, 3)
    ax.imshow(np.reshape(result[0]*255, (image_size, image_size)), cmap="gray")
